<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 829ms/step - loss: 0.6759 - accuracy: 0.5730 - val_loss: 0.7118 - val_accuracy: 0.4341
Epoch 2/10
26/26 [==============================] - 21s 803ms/step - loss: 0.6423 - accuracy: 0.6338 - val_loss: 0.6603 - val_accuracy: 0.4927
Epoch 3/10
26/26 [==============================] - 21s 800ms/step - loss: 0.5908 - accuracy: 0.7032 - val_loss: 0.5582 - val_accuracy: 0.8195
Epoch 4/10
26/26 [==============================] - 21s 800ms/step - loss: 0.5559 - accuracy: 0.7567 - val_loss: 0.7020 - val_accuracy: 0.5220
Epoch 5/10
26/26 [==============================] - 20s 785ms/step - loss: 0.4754 - accuracy: 0.7908 - val_loss: 0.4360 - val_accuracy: 0.8390
Epoch 6/10
26/26 [==============================] - 21s 789ms/step - loss: 0.4283 - accuracy: 0.8114 - val_loss: 0.4244 - val_accuracy: 0.8537
Epoch 7/10
26/26 [==============================] - 20s 784ms/step - loss: 0.3767 - accuracy: 0.8418 - val_loss: 0.3674 - val_accuracy: 0.8829

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.58.h5
26/26 - 21s - loss: 0.6404 - accuracy: 0.6302 - val_loss: 0.5815 - val_accuracy: 0.7366
Epoch 2/5

Epoch 00002: saving model to weights.02-0.51.h5
26/26 - 20s - loss: 0.5684 - accuracy: 0.7275 - val_loss: 0.5096 - val_accuracy: 0.7707
Epoch 3/5

Epoch 00003: saving model to weights.03-0.45.h5
26/26 - 20s - loss: 0.4978 - accuracy: 0.7555 - val_loss: 0.4543 - val_accuracy: 0.7854
Epoch 4/5

Epoch 00004: saving model to weights.04-0.38.h5
26/26 - 20s - loss: 0.4287 - accuracy: 0.7932 - val_loss: 0.3764 - val_accuracy: 0.8390
Epoch 5/5

Epoch 00005: saving model to weights.05-0.35.h5
26/26 - 20s - loss: 0.3783 - accuracy: 0.8406 - val_loss: 0.3544 - val_accuracy: 0.8829


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6834 - accuracy: 0.5365 - val_loss: 0.6816 - val_accuracy: 0.5415


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6706 - accuracy: 0.6046 - val_loss: 0.6883 - val_accuracy: 0.4683
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6334 - accuracy: 0.6460 - val_loss: 0.6110 - val_accuracy: 0.7220


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6723 - accuracy: 0.5681 - val_loss: 0.6862 - val_accuracy: 0.4488
Epoch 2/50
26/26 - 20s - loss: 0.6072 - accuracy: 0.6922 - val_loss: 0.5864 - val_accuracy: 0.7073
Epoch 3/50
26/26 - 20s - loss: 0.5697 - accuracy: 0.7433 - val_loss: 0.5120 - val_accuracy: 0.8537
Epoch 4/50
26/26 - 20s - loss: 0.4837 - accuracy: 0.8041 - val_loss: 0.4427 - val_accuracy: 0.8488
Epoch 5/50
26/26 - 20s - loss: 0.4181 - accuracy: 0.8455 - val_loss: 0.4153 - val_accuracy: 0.8390
Epoch 6/50
26/26 - 21s - loss: 0.3742 - accuracy: 0.8516 - val_loss: 0.2910 - val_accuracy: 0.9317
Epoch 7/50
26/26 - 20s - loss: 0.3030 - accuracy: 0.9027 - val_loss: 0.2407 - val_accuracy: 0.9610
Epoch 8/50
26/26 - 20s - loss: 0.2241 - accuracy: 0.9440 - val_loss: 0.1899 - val_accuracy: 0.9561
Epoch 9/50
26/26 - 21s - loss: 0.1909 - accuracy: 0.9428 - val_loss: 0.1625 - val_accuracy: 0.9463
Epoch 10/50
26/26 - 20s - loss: 0.1354 - accuracy: 0.9757 - val_loss: 0.1355 - val_accuracy: 0.9659
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 847ms/step - loss: 0.6652 - accuracy: 0.6302 - val_loss: 0.7025 - val_accuracy: 0.4341
Epoch 2/5
26/26 [==============================] - 20s 775ms/step - loss: 0.6046 - accuracy: 0.7178 - val_loss: 0.6032 - val_accuracy: 0.6488
Epoch 3/5
26/26 [==============================] - 20s 777ms/step - loss: 0.5477 - accuracy: 0.7555 - val_loss: 0.4833 - val_accuracy: 0.8537
Epoch 4/5
26/26 [==============================] - 21s 796ms/step - loss: 0.4545 - accuracy: 0.8248 - val_loss: 0.4146 - val_accuracy: 0.8683
Epoch 5/5
26/26 [==============================] - 20s 785ms/step - loss: 0.4064 - accuracy: 0.8504 - val_loss: 0.3397 - val_accuracy: 0.9024


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.630170,0.666309,0.434146,0.702538
1,1,0.717762,0.605889,0.648780,0.603227
2,2,0.755474,0.549139,0.853659,0.483300
3,3,0.824818,0.455282,0.868293,0.414631
4,4,0.850365,0.407466,0.902439,0.339667


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6781 - accuracy: 0.5462 - val_loss: 0.6776 - val_accuracy: 0.4927

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 795ms/step - loss: 0.6619 - accuracy: 0.5669 - val_loss: 0.6616 - val_accuracy: 0.7171

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6522 - accuracy: 0.6971 - val_loss: 0.6677 - val_accuracy: 0.5024

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 789ms/step - loss: 0.6479 - accuracy: 0.6010 - val_loss: 0.6615 - val_accuracy: 0.5463

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 784ms/step - loss: 0.6457

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 827ms/step - loss: 0.6819 - accuracy: 0.5572 - val_loss: 0.6838 - val_accuracy: 0.4439
Epoch 2/50
26/26 [==============================] - 20s 780ms/step - loss: 0.6584 - accuracy: 0.6229 - val_loss: 0.6633 - val_accuracy: 0.5024
Epoch 3/50
26/26 [==============================] - 20s 788ms/step - loss: 0.6303 - accuracy: 0.7153 - val_loss: 0.6162 - val_accuracy: 0.7854
Epoch 4/50
26/26 [==============================] - 21s 795ms/step - loss: 0.5854 - accuracy: 0.7895 - val_loss: 0.5520 - val_accuracy: 0.8439
Epoch 5/50
26/26 [==============================] - 20s 770ms/step - loss: 0.5278 - accuracy: 0.8151 - val_loss: 0.5089 - val_accuracy: 0.7951
Epoch 6/50
26/26 [==============================] - 20s 773ms/step - loss: 0.4575 - accuracy: 0.8200 - val_loss: 0.4605 - val_accuracy: 0.8000
Epoch 7/50
26/26 [==============================] - 21s 792ms/step - loss: 0.3881 - accuracy: 0.8686 - val_loss: 0.3288 - val_accuracy: 0.9415

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4598), started 0:16:59 ago. (Use '!kill 4598' to kill it.)